<a href="https://colab.research.google.com/github/abdulahad89/Encoder-Decoder-based-Translation-Model/blob/main/AR_SR_seq2seq_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install AudioSegment

     |████████████████████████████████| 66 kB 3.3 MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72373 sha256=241f883c6d2fc35a49df0c9ac204501f08542f9ec94a7c69f1b38929e03ebe83
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
Successfully built webrtcvad


In [2]:
import pandas as pd
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
from scipy.io import wavfile
import warnings
import glob 
from pydub import AudioSegment

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('drive/My Drive/Sajid_Project_Grading/SR/arabic/full_data.csv')

In [5]:
data.head(10)

,Unnamed: 0,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,0,e23fcbfdbd4639b11c95fb4c8fb94d4b8c576234203af2...,common_voice_ar_19321578.mp3,.أقدّر شجاعته,0,0,twenties,male,NaN,ar,NaN
1,1,f30a72a1cb9deeded2f85b1871b1c7bc1ba3881dd4b765...,common_voice_ar_19341226.mp3,.أعرفها,1,1,twenties,male,NaN,ar,NaN
2,2,6de14f28f77ba488714e21cd47dde88930a6922232b1cb...,common_voice_ar_19346566.mp3,ما الذي كنت تفعله آن ذاك ؟,1,0,twenties,female,NaN,ar,NaN
3,3,e23fcbfdbd4639b11c95fb4c8fb94d4b8c576234203af2...,common_voice_ar_19356833.mp3,وقع الرجل العجوز على الأرض.,1,0,twenties,male,NaN,ar,NaN
4,4,a03a946ee0bfa3c6f35cfa4ba7566da913ec3b828d8feb...,common_voice_ar_19376298.mp3,.يمكنها تحدث اليابانيّة,1,0,thirties,male,NaN,ar,NaN
5,5,f38eaaf535a8b01ee2fc6a673f91499fa59a508a3dbdda...,common_voice_ar_19384302.mp3,تتألف هذه النظرية من ثلاثة أجزاء.,0,1,thirties,male,NaN,ar,NaN
6,6,61ca692e975b8e4882cdfcb40d756ab2b8e9eeeb6b9fb9...,common_voice_ar_19398440.mp3,لا يمكنني إعطائك جواباً نهائياً اليوم.,1,1,NaN,NaN,NaN,ar,NaN
7,7,61ca692e975b8e4882cdfcb40d756ab2b8e9eeeb6b9fb9...,common_voice_ar_19408158.mp3,علمتني عمتي كيف أحضّر قهوةً لذيذة.,0,0,NaN,NaN,NaN,ar,NaN
8,8,61ca692e975b8e4882cdfcb40d756ab2b8e9eeeb6b9fb9...,common_voice_ar_19408245.mp3,.فبدأ الطبيب بالكشف عليها,1,0,NaN,NaN,NaN,ar,NaN
9,9,a4a76115535203d52f6ffd146fdec48a0363deb9e05628...,common_voice_ar_19421231.mp3,لا تبعد المحطة كثيراً عن هنا.,0,1,NaN,NaN,NaN,ar,NaN


In [6]:
path_2 = 'drive/My Drive/Sajid_Project_Grading/SR/arabic/clips_wav'
all_filenames = []
for filename in os.listdir(path_2):
    if filename.endswith(".wav"):
        all_filenames.append([filename[:-4]+'.mp3',filename])

In [7]:
len(all_filenames)

2087

In [8]:
file = pd.DataFrame(data = all_filenames, columns = ['path','name'] )

In [9]:
new_clipped = pd.merge(file,data,on = ['path'], how = 'inner')

In [10]:
len(new_clipped)

2087

In [11]:
new_clipped 

,path,name,Unnamed: 0,client_id,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,common_voice_ar_22023014.mp3,common_voice_ar_22023014.wav,4716,179de31290404c21cb681e479c76824f56b3539a400175...,.صوتها ناعم و واضح,0,0,twenties,male,NaN,ar,NaN
1,common_voice_ar_22023020.mp3,common_voice_ar_22023020.wav,4718,179de31290404c21cb681e479c76824f56b3539a400175...,آه ، في الواقع,0,0,twenties,male,NaN,ar,NaN
2,common_voice_ar_22023019.mp3,common_voice_ar_22023019.wav,4719,179de31290404c21cb681e479c76824f56b3539a400175...,أسرع يا مارتن. لقد تأخّرنا.,0,0,twenties,male,NaN,ar,NaN
3,common_voice_ar_22023103.mp3,common_voice_ar_22023103.wav,4720,179de31290404c21cb681e479c76824f56b3539a400175...,.تزداد الحوادث سنةً بعد سنة,0,0,twenties,male,NaN,ar,NaN
4,common_voice_ar_22023107.mp3,common_voice_ar_22023107.wav,4721,179de31290404c21cb681e479c76824f56b3539a400175...,.ربما ستمطر الليلة,0,0,twenties,male,NaN,ar,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2082,common_voice_ar_21595045.mp3,common_voice_ar_21595045.wav,3711,3411d26e14d4532b4cb0d6aa9638c8c8136c2804620a55...,الحب أعمى.,0,0,thirties,male,NaN,ar,NaN
2083,common_voice_ar_21595048.mp3,common_voice_ar_21595048.wav,3715,3411d26e14d4532b4cb0d6aa9638c8c8136c2804620a55...,لولا نصائحك لفشلت.,0,0,thirties,male,NaN,ar,NaN
2084,common_voice_ar_21595046.mp3,common_voice_ar_21595046.wav,3713,3411d26e14d4532b4cb0d6aa9638c8c8136c2804620a55...,لدي القليل من المال.,0,0,thirties,male,NaN,ar,NaN
2085,common_voice_ar_21595049.mp3,common_voice_ar_21595049.wav,3714,3411d26e14d4532b4cb0d6aa9638c8c8136c2804620a55...,دائماً ما تتوقع حدوث المشاكل.,0,0,thirties,male,NaN,ar,NaN


In [ ]:
"""aud_data = []
path_2 = 'drive/My Drive/Sajid_Project_Grading/SR/arabic/clips_wav'
for wav in new_clipped['name']:
    samples , sample_rate = librosa.load(path_2 +'/'+ wav,16000) 
    samples = librosa.resample(samples, sample_rate, 8000)
    aud_data.append(samples)"""

In [ ]:
import numpy as np
aud_data = []
for wav in new_clipped['name']:
    samples , sample_rate = librosa.load(path_2 +'/'+ wav[:-4] + '.wav',16000) 
    x = librosa.feature.mfcc(samples, sr=16000)
    x = np.transpose(np.array(x))
    aud_data.append(x)

In [ ]:
input = np.asarray(aud_data)

In [ ]:
input[8].shape

In [ ]:
input

In [ ]:
max_input_len = max(len(sen) for sen in input)

In [ ]:
max_input_len

In [ ]:
print(aud_data[25])

In [ ]:
import os, sys
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
sen_len = new_clipped['sentence'].apply(lambda x: len(x.split(' ')))
max(sen_len)

In [ ]:
BATCH_SIZE = 32
EPOCHS = 16
LSTM_NODES =256
#Using only 600 out of 1087
NUM_SENTENCES = 2087
MAX_SENTENCE_LENGTH = max(sen_len)
MAX_NUM_WORDS = 5000
EMBEDDING_SIZE = 50


In [ ]:
output_sentences = []
output_sentences_inputs = []

count = 0
#Using only 600 out of 1087
for i in range(2087):
    count += 1

    if count > NUM_SENTENCES:
        break

    output = new_clipped.iloc[i,4]

    output_sentence = output + ' <eos>'
    output_sentence_input = '<sos> ' + output

    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentence_input)


In [ ]:
#Tokenizing output sequences and 1 palce shifted o/p sequences as an input to o/p 
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)

In [ ]:
output_tokenizer.word_index

In [ ]:
#Padding of i/p, o/p, i/p to o/p sequences
encoder_input_sequences = pad_sequences(input, maxlen=max_input_len)     #Using only 600 out of 1087
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')

In [ ]:
#encoder_input_sequences = encoder_input_sequences.reshape(200,1,67008)

In [ ]:
encoder_input_sequences.shape

In [ ]:
decoder_targets_one_hot = np.zeros((
        2087,
        max_out_len,
        num_words_output
    )
)

In [ ]:
decoder_targets_one_hot.shape

for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [ ]:
#encoder_inputs_placeholder = Input(shape=(161,1,max_input_len,))

In [ ]:
#encoder_inputs_placeholder.shape

In [ ]:
#Encoder part      
encoder_inputs_placeholder = Input(shape=(313,20))
#x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

#encoder states
encoder_outputs, h, c = encoder(encoder_inputs_placeholder)
encoder_states = [h, c]


In [ ]:
encoder_inputs_placeholder.shape

In [ ]:
#Decoder part
decoder_inputs_placeholder = Input(shape=(max_out_len,))
decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)
decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)


In [ ]:
#Dense layer for softmax (final layer)
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [ ]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=32,
    epochs=16,
    validation_split=0.1,
)
  

In [ ]:
model.save('drive/My Drive/Sajid_Project_Grading/SR/model.h5')

In [ ]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [ ]:
#idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [ ]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [ ]:
#|TESTIMG MODEL
i = np.random.choice(len(input))
input_seq = encoder_input_sequences[i]
#print(input_seq.shape)
input_seq = input_seq.reshape(1,313,20)
text = translate_sentence(input_seq)
print('-')
#print(i)
#print('Input:', input_sentences[i])
print('Response:', text)


In [ ]:
i

In [ ]:
data.iloc[i,3]

In [ ]:
############testing Audio ##################

In [ ]:
path_test = 'drive/My Drive/Sajid_Project_Grading/SR/arabic/test_audio/output.wav'

In [ ]:
samples_test , sample_rate_test = librosa.load(path_test,16000) 
x_test = librosa.feature.mfcc(samples_test, sr=16000)
x_test = np.array(x_test)
input_test = np.asarray(x_test)

In [ ]:
encoder_input_sequences_test = pad_sequences(input_test, maxlen=max_input_len)     

In [ ]:
encoder_input_sequences_test = np.transpose(encoder_input_sequences_test)

In [ ]:
input_seq = encoder_input_sequences_test
#print(input_seq.shape)
input_seq = input_seq.reshape(1,262,20)
text = translate_sentence(input_seq)
print('-')
#print(i)
#print('Input:', input_sentences[i])
print('Response:', text)

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

In [ ]:
!pip install pyaudio